# Running stanford_alpaca on Amazon SageMaker

This is a sample code to run stanford_alpaca on Amazon SageMaker, for demo or research use only!

This sample we will show how to configure with **FSx for lustre**, FSx is a high performance storage service, could be mount to training instances, easy to use, suitable for large mount dataset (> hundreds GB), you also could use FSx to store your checkpoint and model files.

In [ ]:
## Update sagemaker python sdk version
!pip install -U sagemaker

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

In [ ]:
!rm -rf src
!mkdir src

In [ ]:
## download training script from github
!cd src && git clone https://github.com/tatsu-lab/stanford_alpaca.git

In [ ]:
## check data
!head -n 44 src/stanford_alpaca/alpaca_data.json

**Generate sample dataset for debug use.**

In [ ]:
%%writefile src/sample_dataset.json
[
    {
        "instruction": "Give three tips for staying healthy.",
        "input": "",
        "output": "1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule."
    },
    {
        "instruction": "What are the three primary colors?",
        "input": "",
        "output": "The three primary colors are red, blue, and yellow."
    },
    {
        "instruction": "Describe the structure of an atom.",
        "input": "",
        "output": "An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom."
    },
    {
        "instruction": "How can we reduce air pollution?",
        "input": "",
        "output": "There are a number of ways to reduce air pollution, such as shifting to renewable energy sources, encouraging the use of public transportation, prohibiting the burning of fossil fuels, implementing policies to reduce emissions from industrial sources, and implementing vehicle emissions standards. Additionally, individuals can do their part to reduce air pollution by reducing car use, avoiding burning materials such as wood, and changing to energy efficient appliances."
    },
    {
        "instruction": "Describe a time when you had to make a difficult decision.",
        "input": "",
        "output": "I had to make a difficult decision when I was working as a project manager at a construction company. I was in charge of a project that needed to be completed by a certain date in order to meet the client\u2019s expectations. However, due to unexpected delays, we were not able to meet the deadline and so I had to make a difficult decision. I decided to extend the deadline, but I had to stretch the team\u2019s resources even further and increase the budget. Although it was a risky decision, I ultimately decided to go ahead with it to ensure that the project was completed on time and that the client\u2019s expectations were met. The project was eventually successfully completed and this was seen as a testament to my leadership and decision-making abilities."
    },
    {
        "instruction": "Identify the odd one out.",
        "input": "Twitter, Instagram, Telegram",
        "output": "Telegram"
    },
    {
        "instruction": "Explain why the following fraction is equivalent to 1/4",
        "input": "4/16",
        "output": "The fraction 4/16 is equivalent to 1/4 because both numerators and denominators are divisible by 4. Dividing both the top and bottom numbers by 4 yields the fraction 1/4."
    },
    {
        "instruction": "Write a short story in third person narration about a protagonist who has to make an important career decision.",
        "input": "",
        "output": "John was at a crossroads in his life. He had just graduated college and was now facing the big decision of what career to pursue. After much deliberation, he decided that he wanted to be an accountant and help the financially disadvantaged. He had always been good with numbers and enjoyed seeing the tangible results of his work. \n\nJohn enrolled in accounting courses and initially found it quite challenging. He had to learn multiple systems and regulations quickly, but he worked hard and eventually excelled in his studies. After a few years, John started working at an accounting firm in his city. He was eager to put his knowledge of taxes and accounting to use in a real-world setting.\n\nJohn loved his job, as it let him express his creativity in finding strategies to save his clients money. After a few years at the firm, he became a senior accountant and was asked to manage bigger and more challenging cases. He was now a respected figure in the financial industry, but he still remembers when he was just a recent college graduate, unsure of the direction in which his life would take him."
    }
]

## Download pretrained model from HuggingFace Hub

To avoid download model from Huggingface hub failure, we download first and push those model files to S3 bucket first.

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "decapoda-research/llama-7b-hf"#

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
)

**Upload model files to S3**

In [ ]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')
for root, dirs, files in paths:
    for file in files:
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

In [ ]:
%%script env sagemaker_default_bucket=$sagemaker_default_bucket local_model_path=$local_model_path bash

chmod +x ./s5cmd
./s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/llama/pretrain/7B/

rm -rf model

## Prepare a docker image

In [ ]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

COPY stanford_alpaca/requirements.txt ./
RUN python3 -m pip install -r requirements.txt

## Install transfomers version which support LLaMaTokenizer
RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@68d640f7c368bcaaaecfc678f11908ebbd3d6176

RUN pip3 uninstall -y deepspeed && pip3 install deepspeed

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

### ECR Login (Must run before docker build)

In [ ]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

**Build image and push to ECR.**

In [ ]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-alpaca-demo"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### Modify Deepspeed config to save model properply.

We will set ```stage3_gather_16bit_weights_on_model_save``` to ```Ture```.

In [ ]:
import json

ds_config_file = './src/stanford_alpaca/configs/default_offload_opt_param.json'
with open (ds_config_file, 'rb') as f:
    ds_config = json.load(f)
    f.close()
    
ds_config['zero_optimization']['stage3_gather_16bit_weights_on_model_save'] = True

with open(ds_config_file, 'w') as f:
    json.dump(ds_config, f, indent=2)
    f.close()

### Generate training entrypoint script

**Note: DO NOT CHANGE BELOW VAlUE OF "output_dir" and "cache_dir", keep it "/tmp/llama_out" and "/tmp".**

Below is just a testing to fine-tune on a sample dataset (just 8 samples), you could change ```data_path``` to your dataset for furthur fine tune. As we will use FSx in this sample, the ```data_pat``` should be the path where FSx mounted to training instances, such as ```/opt/ml/input/data/{CHANNEL_NAME}```.

In this sample, we use ```/opt/ml/input/data/fsx```, your dataset should be stored in FSx first.

In [ ]:
%%writefile src/train.sh
#!/bin/bash

chmod +x ./s5cmd
./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/7B/* /tmp/llama_pretrain/

torchrun --nproc_per_node=8 --master_port=12345 stanford_alpaca/train.py \
    --model_name_or_path "/tmp/llama_pretrain/" \
    --data_path /opt/ml/input/data/fsx/alpaca_data.json \
    --bf16 True \
    --output_dir "/tmp/llama_out" \
    --num_train_epochs 3 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 8 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 2000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --deepspeed "./stanford_alpaca/configs/default_offload_opt_param.json" \
    --tf32 True \
    --cache_dir '/tmp' \
    --report_to "none"

if [ $? -eq 1 ]; then
    echo "Training script error, please check CloudWatch logs"
    exit 1
fi

./s5cmd sync /tmp/llama_out s3://$MODEL_S3_BUCKET/llama/output/$(date +%Y-%m-%d-%H-%M-%S)/

In [ ]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

<!-- ### Modify train.py a little about how to save model

Modify the model save methods in training script, change from 

```
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=training_args.output_dir)
```

to

```
tokenizer.save_pretrained(training_args.output_dir)
trainer.save_model(training_args.output_dir)
``` -->

**The modified training script**

Everything is ready, let's launch the training job.

## Create SageMaker Training Job

**Before run below cell, you should alread setup FSx in FSx console.**

In [ ]:
## fsx integrate

from sagemaker.inputs import FileSystemInput

# Specify FSx Lustre file system id.
file_system_id = "fs-xxxxxxxxxx" # Change to your Fsx FS id

# Specify directory path for input data on the file system. 
# You need to provide normalized and absolute path below.
file_system_directory_path = '/oablmlcdev' # Change to your Fsx Mount name which is given in FSx FS details

# Specify the access mode of the mount of the directory associated with the file system. 
file_system_access_mode = 'rw'

# Specify your file system type.
file_system_type = 'FSxLustre'

fsx_fs = FileSystemInput(file_system_id=file_system_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)

fsx_channels = {'fsx': fsx_fs}

### Notice
Before run below code, make sure you have :
- Config VPC endpoint for S3, and add related route to below subnet you used
- Config VPC NAT Gateway (if you need pip install during the training or download from internet
    - Add route(0.0.0.0/0 through NAT GW) to route table which is used by below subnet you used
- Add route **allow ALL Traffic, source is the security group itself** to route table which is used by below subnet you used

In [ ]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'stanford-alpaca-demo'         

instance_type = 'ml.p4d.24xlarge'

estimator = Estimator(role=role,
                      entry_point='train.sh',
                      source_dir='./src',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      subnets=['subnet-xxxxxxx'], # Should be same vpc with FSx, best to use same subnet with FSx
                      security_group_ids=['sg-xxxxxxxx'], # Needed when use FSx
                      keep_alive_period_in_seconds=60*30, # Optional to set, Recommend use when debug and fast to relaunch without provision instances and images download, need submit warm pool instances limit increase first
                      disable_profiler=True,
                      debugger_hook_config=False)

estimator.fit(inputs=fsx_channels)

## Reference

[SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)

[DeepSpeed Configuration JSON](https://www.deepspeed.ai/docs/config-json/)

[SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples)